In [1]:
import pandas as pd 
import requests
from pathlib import Path

# USA Balancing Authority Interchnage Capacity
This notebook estimates tranmsission capacity between balancing authorities in the USA

## Helper Functions 

In [2]:
def download_file(url: str, destination: str):
    response = requests.get(url)
    
    if response.status_code == 200:
        with open(destination, "wb") as file:
            file.write(response.content)
        print(f"File downloaded successfully to {destination}")
    else:
        print(f"Failed to download file. Status code: {response.status_code}")

In [3]:
def file_exists(file_path: str):
    path = Path(file_path)
    return path.is_file()

In [4]:
def create_directory(directory: str):
    d = Path(directory)
    if not d.exists():
        d.mkdir(parents=True)
        print(f"Directory '{d}' created")

## Extract flow data from EIA
Downloaded 6-month files on interchanges for 2019-2023 from [EIA930](https://www.eia.gov/electricity/gridmonitor/dashboard/electric_overview/US48/US48)

In [5]:
# Note: Sometimes the download on this gets stuck and idk why. 
# If needed, just try rerunning a couple times. It should only take ~10sec per file download

files = [f"{y}_{m}" for y in range(2019,2024) for m in ["Jan_Jun", "Jul_Dec"]]

create_directory("flows")

base_url = "https://www.eia.gov/electricity/gridmonitor/sixMonthFiles/EIA930_INTERCHANGE"
for f in files: 
    if not file_exists(f"flows/{f}.csv"):
        url = f"{base_url}_{f}.csv"
        print(url)
        download_file(url, f"flows/{f}.csv")

https://www.eia.gov/electricity/gridmonitor/sixMonthFiles/EIA930_INTERCHANGE_2019_Jul_Dec.csv
File downloaded successfully to flows/2019_Jul_Dec.csv
https://www.eia.gov/electricity/gridmonitor/sixMonthFiles/EIA930_INTERCHANGE_2021_Jul_Dec.csv
File downloaded successfully to flows/2021_Jul_Dec.csv


## Import Data
Note, this dataframe will load about 1GB of data into memory 

In [6]:
dtypes = {
    "Balancing Authority":str,
    "Data Date":str,
    "Hour Number":int,
    "Directly Interconnected Balancing Authority":str,
    "Interchange (MW)":str,
    "Local Time at End of Hour":str,
    "UTC Time at End of Hour":str,
    "Region":str,
    "DIBA_Region":str
}

dfs = []
for f in files:
    df = pd.read_csv(f"flows/{f}.csv", dtype=dtypes)
    df["Interchange (MW)"] = df["Interchange (MW)"].astype(str)
    df["Interchange (MW)"] = df["Interchange (MW)"].str.replace(",","")
    df["Interchange (MW)"] = df["Interchange (MW)"].astype(float)
    dfs.append(df)
df = pd.concat(dfs).reset_index(drop=True)
df.head()

,Balancing Authority,Data Date,Hour Number,Directly Interconnected Balancing Authority,Interchange (MW),Local Time at End of Hour,UTC Time at End of Hour,Region,DIBA_Region
0,AEC,01/01/2019,1,MISO,28.0,01/01/2019 1:00:00 AM,01/01/2019 7:00:00 AM,SE,MIDW
1,AEC,01/01/2019,1,SOCO,25.0,01/01/2019 1:00:00 AM,01/01/2019 7:00:00 AM,SE,SE
2,AEC,01/01/2019,2,MISO,33.0,01/01/2019 2:00:00 AM,01/01/2019 8:00:00 AM,SE,MIDW
3,AEC,01/01/2019,2,SOCO,-1.0,01/01/2019 2:00:00 AM,01/01/2019 8:00:00 AM,SE,SE
4,AEC,01/01/2019,3,MISO,34.0,01/01/2019 3:00:00 AM,01/01/2019 9:00:00 AM,SE,MIDW


### Drop Retired BAs 
Manually taken from the list of reporting entities provided here by the EIA [here](https://www.eia.gov/electricity/gridmonitor/about) 

In [7]:
retired = ["AEC", "EEI", "GLHB", "GRMA", "NSB", "OVEC", "WAUE", "CFE"]
df = df[
    (~df["Balancing Authority"].isin(retired)) & 
    (~df["Directly Interconnected Balancing Authority"].isin(retired)).dropna()
]

## Format Data
Get timeseries of formatted flows to/from each region

In [8]:
df["From_raw"] = df["Region"].str.cat(df["Balancing Authority"], sep="-")
df["To_raw"] = df["DIBA_Region"].str.cat(df["Directly Interconnected Balancing Authority"], sep="-")
df.head()

,Balancing Authority,Data Date,Hour Number,Directly Interconnected Balancing Authority,Interchange (MW),Local Time at End of Hour,UTC Time at End of Hour,Region,DIBA_Region,From_raw,To_raw
8686,AECI,01/01/2019,1,MISO,333.0,01/01/2019 1:00:00 AM,01/01/2019 7:00:00 AM,MIDW,MIDW,MIDW-AECI,MIDW-MISO
8687,AECI,01/01/2019,1,SPA,-29.0,01/01/2019 1:00:00 AM,01/01/2019 7:00:00 AM,MIDW,CENT,MIDW-AECI,CENT-SPA
8688,AECI,01/01/2019,1,SWPP,-503.0,01/01/2019 1:00:00 AM,01/01/2019 7:00:00 AM,MIDW,CENT,MIDW-AECI,CENT-SWPP
8689,AECI,01/01/2019,1,TVA,195.0,01/01/2019 1:00:00 AM,01/01/2019 7:00:00 AM,MIDW,TEN,MIDW-AECI,TEN-TVA
8690,AECI,01/01/2019,2,MISO,457.0,01/01/2019 2:00:00 AM,01/01/2019 8:00:00 AM,MIDW,MIDW,MIDW-AECI,MIDW-MISO


In [9]:
# correct for the negative flows 
# the data can be +/- for MISO->ASCI and ASCI->MISO 
# correct everything so all flows are positive by flipping BA flow paths 
df["From"] = df.apply(lambda x: x["From_raw"] if x["Interchange (MW)"] >= 0 else x["To_raw"], axis=1)
df["To"] = df.apply(lambda x: x["To_raw"] if x["Interchange (MW)"] >= 0 else x["From_raw"], axis=1)
df["Capacity (MW)"] = df["Interchange (MW)"].abs()
df.head()

,Balancing Authority,Data Date,Hour Number,Directly Interconnected Balancing Authority,Interchange (MW),Local Time at End of Hour,UTC Time at End of Hour,Region,DIBA_Region,From_raw,To_raw,From,To,Capacity (MW)
8686,AECI,01/01/2019,1,MISO,333.0,01/01/2019 1:00:00 AM,01/01/2019 7:00:00 AM,MIDW,MIDW,MIDW-AECI,MIDW-MISO,MIDW-AECI,MIDW-MISO,333.0
8687,AECI,01/01/2019,1,SPA,-29.0,01/01/2019 1:00:00 AM,01/01/2019 7:00:00 AM,MIDW,CENT,MIDW-AECI,CENT-SPA,CENT-SPA,MIDW-AECI,29.0
8688,AECI,01/01/2019,1,SWPP,-503.0,01/01/2019 1:00:00 AM,01/01/2019 7:00:00 AM,MIDW,CENT,MIDW-AECI,CENT-SWPP,CENT-SWPP,MIDW-AECI,503.0
8689,AECI,01/01/2019,1,TVA,195.0,01/01/2019 1:00:00 AM,01/01/2019 7:00:00 AM,MIDW,TEN,MIDW-AECI,TEN-TVA,MIDW-AECI,TEN-TVA,195.0
8690,AECI,01/01/2019,2,MISO,457.0,01/01/2019 2:00:00 AM,01/01/2019 8:00:00 AM,MIDW,MIDW,MIDW-AECI,MIDW-MISO,MIDW-AECI,MIDW-MISO,457.0


In [10]:
df["time"] = pd.to_datetime(df["Local Time at End of Hour"])
df = df.set_index("time")[["To", "From", "Capacity (MW)"]]
df.head()

,To,From,Capacity (MW)
time,,,
2019-01-01 01:00:00,MIDW-MISO,MIDW-AECI,333.0
2019-01-01 01:00:00,MIDW-AECI,CENT-SPA,29.0
2019-01-01 01:00:00,MIDW-AECI,CENT-SWPP,503.0
2019-01-01 01:00:00,TEN-TVA,MIDW-AECI,195.0
2019-01-01 02:00:00,MIDW-MISO,MIDW-AECI,457.0


## Extract Max Flow Rates
Get max flow values to/from each region

In [11]:
flows = df.copy()
flows["To-From"] = flows["To"].str.cat(flows["From"], sep=">")
flows["From-To"] = flows["From"].str.cat(flows["To"], sep=">")
flows.head()

,To,From,Capacity (MW),To-From,From-To
time,,,,,
2019-01-01 01:00:00,MIDW-MISO,MIDW-AECI,333.0,MIDW-MISO>MIDW-AECI,MIDW-AECI>MIDW-MISO
2019-01-01 01:00:00,MIDW-AECI,CENT-SPA,29.0,MIDW-AECI>CENT-SPA,CENT-SPA>MIDW-AECI
2019-01-01 01:00:00,MIDW-AECI,CENT-SWPP,503.0,MIDW-AECI>CENT-SWPP,CENT-SWPP>MIDW-AECI
2019-01-01 01:00:00,TEN-TVA,MIDW-AECI,195.0,TEN-TVA>MIDW-AECI,MIDW-AECI>TEN-TVA
2019-01-01 02:00:00,MIDW-MISO,MIDW-AECI,457.0,MIDW-MISO>MIDW-AECI,MIDW-AECI>MIDW-MISO


### Remove outliers
clip anything outside of the 5-95 percentile 

In [12]:
# takes 5-10min to run 
# https://stackoverflow.com/a/43093390
max_flow = {}
for flow in flows["To-From"].unique():
    f = flows[flows["To-From"] == flow]
    q_low = f["Capacity (MW)"].quantile(0.05)
    q_hi  = f["Capacity (MW)"].quantile(0.95)
    f_clipped = f[(f["Capacity (MW)"] < q_hi) & (f["Capacity (MW)"] > q_low)]
    if f_clipped.empty:
        max_flow[flow] = 0
    else:
        max_flow[flow] = f_clipped["Capacity (MW)"].max()
for flow in flows["From-To"].unique():
    f = flows[flows["From-To"] == flow]
    q_low = f["Capacity (MW)"].quantile(0.05)
    q_hi  = f["Capacity (MW)"].quantile(0.95)
    f_clipped = f[(f["Capacity (MW)"] < q_hi) & (f["Capacity (MW)"] > q_low)]
    if f_clipped.empty:
        max_flow[flow] = 0
    else:
        max_flow[flow] = f_clipped["Capacity (MW)"].max()

In [13]:
ba_paths = set(flows["To-From"].unique()).union(flows["From-To"].unique())

capacity = []
for ba_path in ba_paths:
    ba_1 = ba_path.split(">")[0]
    ba_2 = ba_path.split(">")[1]
    try:
        max_flow_to_from = max_flow[f"{ba_1}>{ba_2}"]
    except KeyError:
        max_flow_to_from = 0
    try:
        max_flow_from_to = max_flow[f"{ba_2}>{ba_1}"]
    except KeyError:
        max_flow_from_to = 0
        
    max_flows = sorted([max_flow_to_from, max_flow_from_to])
    
    diff = abs((max_flows[0] - max_flows[1]) / max_flows[1])
    
    if diff < 0.15:
        max_flow_bidrectional = max(max_flow_to_from, max_flow_from_to)
        capacity.append([
            ba_1, ba_2, max_flow_bidrectional, max_flow_bidrectional * (-1)
        ])
    else:
        capacity.append([
            ba_1, ba_2, max_flow_to_from, max_flow_from_to * (-1)
        ])

In [14]:
df_max_flow = pd.DataFrame(capacity, columns=["BA_From", "BA_To", "Cap (MW) +", "Cap (MW) -"])
df_max_flow.head()

,BA_From,BA_To,Cap (MW) +,Cap (MW) -
0,FLA-FPL,FLA-JEA,767.0,-121.0
1,CAR-CPLE,CAR-DUK,1397.0,-1397.0
2,NW-PACW,NW-BPAT,309.0,-2016.0
3,NW-PGE,NW-PACW,167.0,-167.0
4,CAL-LDWP,NW-NEVP,1381.0,-406.0


## Format data for database

In [15]:
region_ba_map = pd.read_csv("./ba-mapper.csv")
region_ba_map.head()

,code,custom_node_code,region-ba,region,ba,Unnamed: 5
0,USABA,USAWE,CAL-BANC,CAL,BANC,1
1,USACI,USAWE,CAL-CISO,CAL,CISO,1
2,USAII,USAWE,CAL-IID,CAL,IID,1
3,USALD,USAWE,CAL-LDWP,CAL,LDWP,1
4,USATI,USAWE,CAL-TIDC,CAL,TIDC,1


In [16]:
region_2_code = region_ba_map.set_index("region-ba").to_dict()["code"]

In [17]:
df_max_flow_formatted = df_max_flow.copy()
df_max_flow_formatted["BA_From"] = df_max_flow_formatted["BA_From"].map(region_2_code)
df_max_flow_formatted["BA_To"] = df_max_flow_formatted["BA_To"].map(region_2_code)
df_max_flow_formatted.head()

,BA_From,BA_To,Cap (MW) +,Cap (MW) -
0,USAFL,USAJE,767.0,-121.0
1,USACE,USADU,1397.0,-1397.0
2,USAPW,USABP,309.0,-2016.0
3,USAPG,USAPW,167.0,-167.0
4,USALD,USANE,1381.0,-406.0


In [18]:
data = []
for _, row in df_max_flow_formatted.iterrows(): # super inefficient :(
    order = sorted([row["BA_From"], row["BA_To"]])
    if row["BA_From"] == order[0]:
        data.append([
            f"TRN{row['BA_From']}{row['BA_To']}",
            row["BA_From"],
            row["BA_To"],
            row["Cap (MW) +"],
            row["Cap (MW) -"],
        ])
    else:
        data.append([
            f"TRN{row['BA_To']}{row['BA_From']}",
            row["BA_To"],
            row["BA_From"],
            row["Cap (MW) -"] * (-1),
            row["Cap (MW) +"] * (-1),
        ])

In [20]:
final = pd.DataFrame(data, columns=["TECHNOLOGY", "From", "To", "Cap (MW) +", "Cap (MW) -"])
final = final.drop_duplicates()
final = final.sort_values(by=["TECHNOLOGY"])
final.to_csv("usa-transmission-capacity.csv", index=False)
final.head()

,TECHNOLOGY,From,To,Cap (MW) +,Cap (MW) -
65,TRNCANABUSANW,CANAB,USANW,194.0,-244.0
120,TRNCANARUSAIS,CANAR,USAIS,732.0,-321.0
9,TRNCANBCUSABP,CANBC,USABP,2136.0,-2136.0
54,TRNCANMBUSAMI,CANMB,USAMI,2086.0,-1210.0
286,TRNCANONUSAMI,CANON,USAMI,1502.0,-818.0
